# Load Libraries

In [275]:
import pandas as pd
import glob
import os

# Read datasets
## NOPD Calls datasets

### Function to Read Multiple CSVs

In [276]:
def read_multiple_csvs_from_folder(folder_path, specific_columns):
  """Reads multiple CSV files from a folder and extracts specific columns.

  Args:
    folder_path: The path to the folder containing the CSV files.
    specific_columns: A list of the specific columns to extract.

  Returns:
    A Pandas DataFrame containing the extracted data.
  """

  # Get a list of all CSV files in the folder.
  csv_files = glob.glob(os.path.join(folder_path + "*.csv"))

  # Create an empty DataFrame to store the extracted data.
  combined_df = pd.DataFrame()

  # Loop through the list of CSV files and extract the specific columns.
  for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df = df[specific_columns]
    combined_df = pd.concat([combined_df, df])

  return combined_df

### Use the function to create a combined dataframe

In [277]:
## First create the folder path
current_working_dir = os.getcwd()
relative_path = "../data/raw-datasets/nopd-calls-csvs/"
folder_path = os.path.join(current_working_dir, relative_path)

## List the columns to extract
specific_columns = ["NOPD_Item", "Type_", "TypeText", "Priority", 
                    "TimeCreate", "Location"]

## Call the function
combined_df = read_multiple_csvs_from_folder(folder_path, specific_columns)

# Create variable for domestic violence and domestic dispute

## Explore the type of calls

In [278]:
## Get the unique list of values in the "TypeText" column
unique_types = combined_df["TypeText"].unique()
## Subset to unique types that contain "DOMESTIC"
unique_type_domestic = [x for x in unique_types if "DOMESTIC" in x]
print(unique_type_domestic)

['DOMESTIC DISTURBANCE', 'DOMESTIC CRIMINAL DA', 'HOMICIDE DOMESTIC', 'SIMPLE BATTERY DOMESTIC', 'CRIMINAL DAMAGE DOMESTIC', 'EXTORTION (THREATS) DOMESTIC', 'AGGRAVATED BATTERY DOMESTIC', 'AGGRAVATED ASSAULT DOMESTIC', 'SIMPLE ASSAULT DOMESTIC', 'SIMPLE BURGLARY DOMESTIC', 'CRIMINAL MISCHIEF DOMESTIC', 'SIMPLE ARSON DOMESTIC', 'AGGRAVATED BURGLARY DOMESTIC', 'DOMESTIC VIOLENCE', 'DOMESTIC DISPUTE']


## Create new variables in the dataset

In [279]:
# Create a new variable that takes 1 if DOMESTIC VIOLENCE, 2 for DOMESTIC DISPUTE and 0 otherwise
combined_df["domestic_violence"] = combined_df["TypeText"].apply(lambda x: 1 if "DOMESTIC VIOLENCE" in x else 2 if "DOMESTIC DISPUTE" in x else 3 if "SIMPLE ASSAULT DOMESTIC" in x else 0)
combined_df["any_domestic"] = combined_df["TypeText"].apply(lambda x: 1 if "DOMESTIC" in x else 0)

In [280]:
combined_df["any_domestic"].value_counts()

any_domestic
0    5432130
1     175737
Name: count, dtype: int64

In [281]:
combined_df["any_domestic"].value_counts(normalize=True)*100

any_domestic
0    96.866242
1     3.133758
Name: proportion, dtype: float64

In [282]:
df_domestic = combined_df[combined_df["any_domestic"] == 1]

In [283]:
df_domestic['TypeText'].value_counts(normalize=True)*100

TypeText
DOMESTIC DISTURBANCE            63.384489
SIMPLE BATTERY DOMESTIC         20.709355
DOMESTIC DISPUTE                 5.080888
CRIMINAL DAMAGE DOMESTIC         2.936775
DOMESTIC VIOLENCE                2.694367
AGGRAVATED ASSAULT DOMESTIC      1.573374
EXTORTION (THREATS) DOMESTIC     1.415183
AGGRAVATED BATTERY DOMESTIC      0.759089
SIMPLE ASSAULT DOMESTIC          0.633902
DOMESTIC CRIMINAL DA             0.542857
SIMPLE BURGLARY DOMESTIC         0.128032
AGGRAVATED BURGLARY DOMESTIC     0.107547
CRIMINAL MISCHIEF DOMESTIC       0.017640
HOMICIDE DOMESTIC                0.009674
SIMPLE ARSON DOMESTIC            0.006828
Name: proportion, dtype: float64

In [284]:
combined_df["domestic_violence"].value_counts()

domestic_violence
0    5593089
2       8929
1       4735
3       1114
Name: count, dtype: int64

In [285]:
combined_df["domestic_violence"].value_counts(normalize=True)*100

domestic_violence
0    99.736477
2     0.159223
1     0.084435
3     0.019865
Name: proportion, dtype: float64

# Clean the Location data

In [286]:
combined_df['Location'].tail()

237060    POINT (-89.96808617 30.02749184)
237061    POINT (-90.03178919 29.98638514)
237062    POINT (-90.00302303 29.93189397)
237063    POINT (-90.06480379 29.94325552)
237064     POINT (-90.0373306 30.00297646)
Name: Location, dtype: object

In [287]:
## Extract the string from the "Location" column that is between the two paranthesis
import re

# Convert the datatype of Location from float to string
combined_df['Location_new'] = str(combined_df['Location'])

# Define a regular expression to match the string between parentheses
regex = r"\(.*?\)"

# Create a new variable to store the extracted string
extracted_string = []

# Iterate over the existing column and extract the string between parentheses
for row in combined_df['Location_new']:
    match = re.search(regex, row)
    if match:
        extracted_string.append(match.group(0))
    else:
        extracted_string.append(None)

# Create a new column and assign the extracted string to it
combined_df['Location_new'] = extracted_string

In [288]:
## Extract the latitude and longitude from the "Location_new" column by splitting the string based on space
combined_df['Longitude'] = combined_df['Location_new'].str.split(' ').str[0]
combined_df['Latitude'] = combined_df['Location_new'].str.split(' ').str[1]

In [289]:
## Remove paranthesis from the "Longitude" and "Latitude" columns
### Longitude
combined_df['Longitude'] = combined_df['Longitude'].str.replace('(', '')
combined_df['Longitude'] = combined_df['Longitude'].str.replace(',', '')

## Latitude
combined_df['Latitude'] = combined_df['Latitude'].str.replace(')', '')
combined_df['Latitude'] = combined_df['Latitude'].str.replace(')', '')

In [290]:
## Remove 'e' from the "Longitude" and "Latitude" columns
### Longitude
combined_df['Longitude'] = combined_df['Longitude'].str.replace('E-', '')
combined_df['Longitude'] = combined_df['Longitude'].str.replace('e-', '')

## Latitude
combined_df['Latitude'] = combined_df['Latitude'].str.replace('E-', '')
combined_df['Latitude'] = combined_df['Latitude'].str.replace('e-', '')

In [291]:
## Convert the datatype of the "Longitude" and "Latitude" columns from string to float
combined_df['Longitude'] = combined_df['Longitude'].astype(float)
combined_df['Latitude'] = combined_df['Latitude'].astype(float)

# Filter cases where Lat or Long data is not in the NOLA range

In [292]:
df_nopd_clean = combined_df[(combined_df['Longitude'].abs() > 10) & (combined_df['Latitude'] > 10)]

In [293]:
df_nopd_clean['Latitude'].max()

nan

In [294]:
combined_df['Latitude'].dtype

dtype('float64')